In [ ]:
import pandas as pd

In [ ]:
# toy example: 4 documents
X_train = [
    'call you tonight',
    'call me a cab',
    'please call me... PLEASE',
    'he called the police'
]
X_train

We will be using countvectorizer to convert text into a matrix of word counts

In [ ]:
# import countvectorizer
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer() # with default parameters

In [ ]:
# "learn the vocabulary"
vect.fit(X_train)

In [ ]:
# exmaine the fitted vocabulary
vect.get_feature_names_out()

In [ ]:
# converting training data into a 'document-term matrix'
X_train_dtm = vect.transform(X_train)
X_train_dtm

In [ ]:
X_train_dtm.toarray()

In [ ]:
pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names_out(), index=X_train)

In [ ]:
X_test = ['please don\'t call me']
X_test_dtm = vect.transform(X_test)
pd.DataFrame(X_test_dtm.toarray(), columns=vect.get_feature_names_out(), index=X_test)

### Tuning the Vectorizer

**stop_words:** Stop words are words like [I, a, an, this, the, ...] that don't add much meaning to a sentence. We can remove them to reduce the number of features.

In [ ]:
vect = CountVectorizer(stop_words='english')
vect.fit(X_train)
vect.get_feature_names_out()

In [ ]:
# list of scikit learn stop words
from sklearn.feature_extraction import _stop_words

sorted(list(_stop_words.ENGLISH_STOP_WORDS))

**ngram_range**: An n-gram is a sequence of n words. For example, "apple juice" is a 2-gram (aka a bigram), and "I love apple juice" is a 4-gram (aka a four-gram). The ngram_range parameter lets us specify the range of n-gram sizes we want to include in our features. In the example above, we included unigrams (ngram_range=(1,1)) and bigrams (ngram_range=(2,2)).

In [ ]:
vect = CountVectorizer(ngram_range=(1, 3)) # 1 grams, 2 grams, 3 grams
vect.fit(X_train)
vect.get_feature_names_out()

**max_df / min_df:** When building the vocabulary, we can set the maximum document frequency (max_df) and minimum document frequency (min_df). If the word frequency is below min_df OR above max_df, the word is ignored. This allows us to exclude words that are too rare or too common to be useful.

In [ ]:
# ignore items that appear in more than 50% of the documents
vect = CountVectorizer(max_df=0.5)
vect.fit(X_train)
vect.get_feature_names_out()

In [ ]:
# only show items that appear in at least 2 documents
vect = CountVectorizer(min_df=2)
vect.fit(X_train)
vect.get_feature_names_out()

### Naive Bayes Classifier

Naive Bayes classifiers are a collection of classification algorithms based on Bayes' Theorem. It is not a single algorithm but a family of algorithms where all of them share a common principle, i.e. every pair of features being classified is independent of each other.

"Naive" assumes that the presence of a particular feature in a class is unrelated to the presence of any other feature. For example, words in a sentence are independent of each other.

It works on the principle of conditional probability. Conditional probability is the probability of something happening, given that something else has already occurred.

Bayes Theorem: 
$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}
$$

Pros:
- Very fast
- Simple to implement


<u>Example</u>

Training documents:

| Column | Document | Label |
| :-- | --: |  --: |
| Doc1 | "basketball is a great game" | not politics |
| Doc2 | "the election is over" | politics |
| Doc3 | "very clean debate" | politics |
| Doc4 | "a close but forgettable race" | not politics |
| Doc5 | "the election is a race" | politics |

Vocabulary: basketball, great, ... , race (ignore stop words)
new document: "a very close race"

<u>goal</u>: predict the label of the new document i.e. to estimate p(politics | "a very close race") or p(not politics | "a very close race")

Naive bayes predicts the label with the Largest Probability

$P(politics | "a .. race") = \frac{1}{p("a .. race")}*p("a .. race" | politics)*p(politics)$ the end is the prior probability, the probability of the label

$P(not-pol | "a .. race") = \frac{1}{p("a .. race")}*p("a .. race" | not-pol)*p(not pol)$ before the prior is the likelihood

$P(politics) = \frac{number\ of\ politics\ docs}{total\ number\ of\ docs} = \frac{3}{5}$

$P(not-politics) = \frac{2}{5}$

Naive bayes:

$P("a...race" | politics) = P("very" | politics) * P("close" | politics) * P("race" | politics)$

$P("very" | politics) = number\ of\ times\ "very"\ appears\ in\ politics\ docs / total\ number\ of\ words\ in\ politics\ docs = 1+\alpha/7+11\alpha$

$P("close" | politics) = 0+\alpha/7+11\alpha$

$P("race" | politics) = 1+\alpha/7+11\alpha$

We dont have "close" in politics docs, so we use Laplace smoothing using an alpha parameter (from 0 to 1) to avoid 0 probability

We can add the alpha parameter to the numerator and the number of words in the vocabulary times alpha to the denominator.